In [7]:
import random

import numpy as np
#import maze as mz

In [94]:
# Description of the maze as a numpy array
maze = np.array([
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 2, 0]
])
# with the convention
# 0 = empty cell
# 1 = obstacle
# 2 = exit of the Maze
# -n = randomly wait n rounds with prob .5
rnd_maze = np.array([
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, -1],
    [0, 1, 1, 1, 1, 1, 0],
    [-6, 0, 0, 0, 0, 2, 0]
])

In [113]:
class MarkoDecisionProcess:
    GOAL_REWARD = 1
    IMPOSSIBLE_REWARD = -1000
    STEP_REWARD = -1

    STAY = 0
    MOVE_LEFT = 1
    MOVE_RIGHT = 2
    MOVE_UP = 3
    MOVE_DOWN = 4

    EMPTY_CELL = 0
    OBSTACLE_CELL = 1
    GOAL_CELL = 2
    RANDOM_CELL = 3

    action_names = {
        STAY: "stay",
        MOVE_LEFT: "move left",
        MOVE_RIGHT: "move right",
        MOVE_UP: "move up",
        MOVE_DOWN: "move down"
    }

    def __init__(self, maze, random_rewards=False):
        self.maze = maze
        self.random_rewards = random_rewards
        self.states, self.map = self.__get_state_dicts()
        self.actions = self.__actions()
        self.n_states = (len(self.states))
        self.n_actions = (len(self.actions))
        self.transition_probs = self.__calc_transition_matrix()
        self.rewards = self.__calc_reward_matrix()

    def __get_state_dicts(self):
        states = {}
        map = {}
        s = 0
        for y in range(self.maze.shape[0]):
            for x in range(self.maze.shape[1]):
                if self.maze[y, x] != 1:
                    states[s] = (y, x)
                    map[(y, x)] = s
                    s += 1
        return states, map

    def __actions(self):
        actions = dict()
        actions[self.STAY] = (0, 0)
        actions[self.MOVE_LEFT] = (0, -1)
        actions[self.MOVE_RIGHT] = (0, 1)
        actions[self.MOVE_UP] = (-1, 0)
        actions[self.MOVE_DOWN] = (1, 0)
        return actions

    def __reward_func(self, s, a, next_s):
        if self.maze[self.states[next_s]] == self.GOAL_CELL:
            return self.GOAL_REWARD
        elif s == next_s and a != self.STAY:  # could not move due to obstacle
            return self.IMPOSSIBLE_REWARD
        elif self.random_rewards and self.maze[self.states[next_s]] < 0:
            stay_for_n_rounds = (1 + abs(self.maze[self.states[next_s]])) * self.STEP_REWARD
            return .5 * self.STEP_REWARD + .5 * stay_for_n_rounds
        else:
            return self.STEP_REWARD

    def __calc_transition_matrix(self):
        dimensions = (self.n_states, self.n_states, self.n_actions)
        transition_probabilities = np.zeros(dimensions)

        for s in range(self.n_states):
            for a in range(self.n_actions):
                next_s = self.next_state(s, a)
                transition_probabilities[next_s, s, a] = 1
        return transition_probabilities

    def __calc_reward_matrix(self):
        rewards = np.zeros((self.n_states, self.n_actions))

        for s in range(self.n_states):
            for a in range(self.n_actions):
                next_s = self.next_state(s, a)
                rewards[s, a] = self.__reward_func(s, a, next_s)
        return rewards

    def next_state(self, s, a):
        (y, x) = self.states.get(s)
        (dy, dx) = self.actions[a]

        # make step -> new = [0-5]
        new_y = y + dy
        new_x = x + dx

        # check bounds, if OOB -> stay
        (yub, xub) = self.maze.shape  # 6,7
        if new_y >= yub or new_y < 0:
            return s
        if new_x >= xub or new_x < 0:
            return s

        # if obstacle -> stay
        if self.maze[new_y][new_x] == 1:
            return s

        return self.map[(new_y, new_x)]

    def dynamic_programming(self, horizon):
        T = horizon

        V = np.zeros((self.n_states, T + 1))
        policy = np.zeros((self.n_states, T + 1))

        # initialize Q function with single step rewards
        Q = np.copy(self.rewards)  # states X actions
        # value function in step T is max reward
        V[:, T] = np.max(Q, axis=1)
        # action with max reward
        policy[:, T] = np.argmax(Q, axis=1)

        for t in range(T - 1, -1, -1):
            for s in range(self.n_states):
                for a in range(self.n_actions):
                    Q[s, a] = self.rewards[s, a] \
                              + np.dot(self.transition_probs[:, s, a], V[:, t + 1])
            V[:, t] = np.max(Q, axis=1)
            policy[:, t] = np.argmax(Q, axis=1)

        return V, policy

    def value_iteration(self, discount=.95, epsilon=.0001):
        V = np.zeros(self.n_states)
        Q = np.zeros((self.n_states, self.n_actions))
        BV = np.zeros(self.n_states)

        tol = epsilon * (1 - discount) / discount

        for s in range(self.n_states):
            for a in range(self.n_actions):
                Q[s, a] = self.rewards[s, a] \
                          + discount * np.dot(self.transition_probs[:, s, a], V)
        BV = np.max(Q, 1)

        delta = np.inf
        while delta > tol:
            V = np.copy(BV)

            for s in range(self.n_states):
                for a in range(self.n_actions):
                    Q[s, a] = self.rewards[s, a] \
                          + discount * np.dot(self.transition_probs[:, s, a], V)
            BV = np.max(Q, 1)

            delta = np.linalg.norm(V - BV)
            print(delta, V)

        policy = np.argmax(Q, 1)
        return V, policy

    def simulate_DP(self, policy, s=0):
        T = policy.shape[1]
        path_s = [s]
        path_a = []

        for t in range(T):
            # get action from policy based on current state and time
            a = policy[s, t]
            path_a.append(a)

            # move next step
            s = mdp.next_state(s, a)
            path_s.append(s)

        for t, s, a in zip(range(T), path_s, path_a):
            print(t, mdp.states[s], mdp.action_names[a])

        return path_s, path_a

    def simulate_VI(self, policy, s=0):
        a = policy[s]
        next_s = mdp.next_state(s, a)

        path_s = [s, next_s]
        path_a = [a]

        while s != next_s:
            s = next_s
            a = policy[s]
            next_s = mdp.next_state(s, a)
            path_s.append(next_s)
            path_a.append(a)

        for s, a in zip(path_s, path_a):
            print(mdp.states[s], mdp.action_names[a])

        return path_s, path_a


In [114]:
time_horizon = 12

mdp = MarkoDecisionProcess(maze)
V, policy = mdp.dynamic_programming(time_horizon)
mdp.simulate_DP(policy)

0 (0, 0) move down
1 (1, 0) move down
2 (2, 0) move down
3 (3, 0) move down
4 (4, 0) move down
5 (5, 0) move right
6 (5, 1) move right
7 (5, 2) move right
8 (5, 3) move right
9 (5, 4) move right
10 (5, 5) stay
11 (5, 5) stay
12 (5, 5) stay


([0, 6, 12, 18, 25, 27, 28, 29, 30, 31, 32, 32, 32, 32],
 [4.0, 4.0, 4.0, 4.0, 4.0, 2.0, 2.0, 2.0, 2.0, 2.0, 0.0, 0.0, 0.0])

In [108]:
V, policy = mdp.value_iteration()
mdp.simulate_VI(policy)

5.539404300103036 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.  1.]
5.262434085097884 [-1.95 -1.95 -1.95 -1.95 -1.95 -1.95 -1.95 -1.95 -1.95 -1.95 -1.95 -1.95
 -1.95 -1.95 -1.95 -1.95 -1.95 -1.95 -1.95 -1.95 -1.95 -1.95 -1.95 -1.95
 -1.95 -1.95 -0.05 -1.95 -1.95 -1.95 -0.05  1.95  1.95  1.95]
4.999312380842989 [-2.8525 -2.8525 -2.8525 -2.8525 -2.8525 -2.8525 -2.8525 -2.8525 -2.8525
 -2.8525 -2.8525 -2.8525 -2.8525 -2.8525 -2.8525 -2.8525 -2.8525 -2.8525
 -2.8525 -2.8525 -2.8525 -2.8525 -2.8525 -2.8525 -1.0475 -2.8525  0.8525
 -2.8525 -2.8525 -1.0475  0.8525  2.8525  2.8525  2.8525]
4.749346761800838 [-3.709875 -3.709875 -3.709875 -3.709875 -3.709875 -3.709875 -3.709875
 -3.709875 -3.709875 -3.709875 -3.709875 -3.709875 -3.709875 -3.709875
 -3.709875 -3.709875 -3.709875 -1.995125 -3.709875 -3.709875 -3.709875
 -3.709875 -3.709875 -1.995125 -0.190125 -3.709875  1.709875 -3.709875
 -1.995125 -0.190125

([0, 6, 12, 18, 25, 27, 28, 29, 30, 31, 32, 32],
 [4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 0])

In [104]:
mdp_rr = MarkoDecisionProcess(rnd_maze, random_rewards=True)
V, policy = mdp_rr.dynamic_programming(12)
mdp_rr.simulate_DP(policy)

0 (1, 0) move down
1 (2, 0) move down
2 (3, 0) move down
3 (4, 0) move down
4 (5, 0) move down
5 (5, 1) move right
6 (5, 2) move right
7 (5, 3) move right
8 (5, 4) move right
9 (5, 5) move right
10 (5, 5) stay
11 (5, 5) stay
12 (5, 5) stay


([0], [4.0, 4.0, 4.0, 4.0, 4.0, 2.0, 2.0, 2.0, 2.0, 2.0, 0.0, 0.0, 0.0])

In [105]:
V, policy = mdp_rr.value_iteration()
mdp_rr.simulate_VI(policy)

5.539404300103036 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.  1.]
5.262434085097884 [-1.95 -1.95 -1.95 -1.95 -1.95 -1.95 -1.95 -1.95 -1.95 -1.95 -1.95 -1.95
 -1.95 -1.95 -1.95 -1.95 -1.95 -1.95 -1.95 -1.95 -1.95 -1.95 -1.95 -1.95
 -1.95 -1.95 -0.05 -1.95 -1.95 -1.95 -0.05  1.95  1.95  1.95]
4.876307443265856 [-2.8525 -2.8525 -2.8525 -2.8525 -2.8525 -2.8525 -2.8525 -2.8525 -2.8525
 -2.8525 -2.8525 -2.8525 -2.8525 -2.8525 -2.8525 -2.8525 -2.8525 -2.8525
 -2.8525 -2.8525 -2.8525 -2.8525 -2.8525 -2.8525 -1.0475 -2.8525  0.8525
 -2.8525 -2.8525 -1.0475  0.8525  2.8525  2.8525  2.8525]
4.572945096032545 [-3.709875 -3.709875 -3.709875 -3.709875 -3.709875 -3.709875 -3.709875
 -3.709875 -3.709875 -3.709875 -3.709875 -3.709875 -3.709875 -3.709875
 -3.709875 -3.709875 -3.709875 -2.495125 -3.709875 -3.709875 -3.709875
 -3.709875 -3.709875 -2.495125 -0.190125 -3.709875  1.709875 -3.709875
 -1.995125 -0.190125

([0, 12, 18, 25, 27, 28, 29, 30, 31, 32, 32],
 [4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 0])